# Objective

The objective of this notebook is to understand the metrics in the sample_tableau_data file of module-progress


* Improve the code by referecing with IDs wherever necessary
* Transfer the code to a dash app
* Remove/Add the labels from the plots where necessary
* Improve dashboard aesthetics

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re


from collections import defaultdict

from datetime import *
import datetime
pio.renderers.default='iframe'

In [2]:
data = pd.read_csv('../data/SAMPLE_module_data.csv')

In [3]:
data.head()

,Unnamed: 0,completed_at,course_id,module_id,items_count,module_name,module_position,state,unlock_at,student_id,student_name,items_id,items_title,items_position,items_indent,items_type,items_module_id,item_cp_req_type,item_cp_req_completed,course_name
0,0,2019-06-25 22:29,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85224,Introduction Text,1,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
1,1,2019-06-25 22:29,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85225,Important Info Moving Foreward,2,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
2,2,2019-06-25 22:29,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85262,Quiz 1 - Past Experience,3,1,Quiz,8135,must_submit,True,Canvas-Sandbox-Course
3,3,NaN,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85263,Designing Data,1,0,Page,8136,must_view,False,Canvas-Sandbox-Course
4,4,NaN,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85228,Talk About It.,2,0,Discussion,8136,must_contribute,True,Canvas-Sandbox-Course


In [4]:
data.columns

Index(['Unnamed: 0', 'completed_at', 'course_id', 'module_id', 'items_count',
       'module_name', 'module_position', 'state', 'unlock_at', 'student_id',
       'student_name', 'items_id', 'items_title', 'items_position',
       'items_indent', 'items_type', 'items_module_id', 'item_cp_req_type',
       'item_cp_req_completed', 'course_name'],
      dtype='object')

In [5]:
categorical_cols = ['course_id',
                   'module_id',
                   'module_name',
                   'state',
                   'student_id',
                   'student_name',
                   'items_id',
                   'items_title',
                   'items_type',
                   'items_module_id',
                   'item_cp_req_type',
                   'item_cp_req_completed',
                   'course_name']

In [6]:
# convert the timestamp to datetime format
# fix the column data types
data['completed_at'] = pd.to_datetime(data['completed_at'])
for col in categorical_cols:
    data[col] = data[col].astype('category')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             420 non-null    int64         
 1   completed_at           103 non-null    datetime64[ns]
 2   course_id              420 non-null    category      
 3   module_id              420 non-null    category      
 4   items_count            420 non-null    int64         
 5   module_name            420 non-null    category      
 6   module_position        420 non-null    int64         
 7   state                  420 non-null    category      
 8   unlock_at              0 non-null      float64       
 9   student_id             420 non-null    category      
 10  student_name           420 non-null    category      
 11  items_id               420 non-null    category      
 12  items_title            420 non-null    category      
 13  items

In [8]:
# Drop the unwanted columns
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data.columns

Index(['completed_at', 'course_id', 'module_id', 'items_count', 'module_name',
       'module_position', 'state', 'unlock_at', 'student_id', 'student_name',
       'items_id', 'items_title', 'items_position', 'items_indent',
       'items_type', 'items_module_id', 'item_cp_req_type',
       'item_cp_req_completed', 'course_name'],
      dtype='object')

## Demo Dashboard

### Tab - Student Module Completion over time

### Dictionaries

In [9]:
# Make a dictionary of module id and module names
module_dict, item_dict, course_dict, student_dict = (defaultdict(str) for _ in range(4))

for _,row in data.iterrows():
    module_dict[str(row['module_id'])] = re.sub(r'^Module\s+\d+:\s+', "", row['module_name'])
    item_dict[str(row['items_module_id'])] = row['items_title']
    course_dict[str(row['course_id'])] = row['course_name']
    student_dict[str(row['student_id'])] = row['student_name']

In [10]:
module_dict.keys()

dict_keys(['8135', '8136', '8137', '8169', '8170', '8171'])

In [11]:
item_dict.keys()

dict_keys(['8135', '8136', '8137', '8169', '8170', '8171'])

In [12]:
course_dict.keys()

dict_keys(['2591'])

In [13]:
student_dict.keys()

dict_keys(['1001', '1002', '1003', '1004', '1005', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1022', '1023', '1024', '1025'])

In [14]:
module_dict.get('8135')

'Course Introduction'

In [15]:
module_ids = list(module_dict.keys())

for module in module_ids:
    # Print Module Status
    # print(module)
    # number of student id related to Module
    data_module = data[(data.module_id.astype(str) == module)]
    # print(data_module.shape)
    # print(data_module['student_id'].unique().size)

    # number of students that have completed the Module
    data_module[data_module.state == 'completed']['student_id'].unique().size

    # number of students that have unlocked the Module 1
    data_module[data_module.state == 'unlocked']['student_id'].unique().size

    # number off students that have started the Module 1
    data_module[data_module.state == 'started']['student_id'].unique().size


    # display
    print(f"Total number of students engaged (start/completed) {module_dict.get(module)}: {data_module['student_id'].unique().size}")
    print(f"Number of students completed {module_dict.get(module)}: {data_module[data_module.state == 'completed']['student_id'].unique().size}")
    
    if (data_module['student_id'].unique().size == 0):
        continue

    print(f"Percentage of completion: {round(data_module[data_module.state == 'completed']['student_id'].unique().size * 100 / data_module['student_id'].unique().size, 1)}%")
    print(f"Percentage of started : {round(data_module[data_module.state == 'started']['student_id'].unique().size * 100 / data_module['student_id'].unique().size, 1)}%")
    print(f"Percentage of unlocked : {round(data_module[data_module.state == 'unlocked']['student_id'].unique().size * 100 / data_module['student_id'].unique().size, 1)}%")
    print("="*100)



Total number of students engaged (start/completed) Course Introduction: 21
Number of students completed Course Introduction: 18
Percentage of completion: 85.7%
Percentage of started : 14.3%
Percentage of unlocked : 0.0%
Total number of students engaged (start/completed) How to Design Data: 21
Number of students completed How to Design Data: 4
Percentage of completion: 19.0%
Percentage of started : 66.7%
Percentage of unlocked : 14.3%
Total number of students engaged (start/completed) How to Design Functions: 21
Number of students completed How to Design Functions: 5
Percentage of completion: 23.8%
Percentage of started : 28.6%
Percentage of unlocked : 47.6%
Total number of students engaged (start/completed) Intro to Object Orientation: 21
Number of students completed Intro to Object Orientation: 3
Percentage of completion: 14.3%
Percentage of started : 14.3%
Percentage of unlocked : 71.4%
Total number of students engaged (start/completed) Data Hierarchies: 21
Number of students complet

In [16]:
## Functions

def get_completed_percentage(df, module,state):
    '''
    returns the state percetage of module in df
    '''
    
    df_module = df[df.module_id.astype(str) == module]
    total_module_students = df_module.student_id.unique().size
    percentage = df_module[df_module.state==state].student_id.unique().size / total_module_students 
    return percentage

In [17]:
## To build a dashboard display with horizontal bars like in Tableau

total_students = data.student_id.unique().size

result = {}
for module in module_dict.keys():
    result[module_dict.get(module)] = [round(get_completed_percentage(data, module, 'unlocked')*100, 1), 
                           round(get_completed_percentage(data, module, 'started')*100, 1),
                           round(get_completed_percentage(data, module, 'completed')*100, 1)]

In [18]:
result

{'Course Introduction': [0.0, 14.3, 85.7],
 'How to Design Data': [14.3, 66.7, 19.0],
 'How to Design Functions': [47.6, 28.6, 23.8],
 'Intro to Object Orientation': [71.4, 14.3, 14.3],
 'Data Hierarchies': [71.4, 14.3, 14.3],
 'Designing Software': [66.7, 33.3, 0.0]}

In [19]:
# convert this to a dataframe 
df = pd.DataFrame(result, index = ['Unlocked', 'Started', 'Completed']).T.reset_index().rename(columns={'index': 'Module'})
df.head()

,Module,Unlocked,Started,Completed
0,Course Introduction,0.0,14.3,85.7
1,How to Design Data,14.3,66.7,19.0
2,How to Design Functions,47.6,28.6,23.8
3,Intro to Object Orientation,71.4,14.3,14.3
4,Data Hierarchies,71.4,14.3,14.3


### Tab - Module Completion Chart

In [20]:
# Melt the DataFrame to convert columns to rows
melted_df = pd.melt(df, id_vars='Module', value_vars=['Unlocked', 'Started', 'Completed'],
                    var_name='Status', value_name='Percentage Completion')


In [21]:
melted_df

,Module,Status,Percentage Completion
0,Course Introduction,Unlocked,0.0
1,How to Design Data,Unlocked,14.3
2,How to Design Functions,Unlocked,47.6
3,Intro to Object Orientation,Unlocked,71.4
4,Data Hierarchies,Unlocked,71.4
5,Designing Software,Unlocked,66.7
6,Course Introduction,Started,14.3
7,How to Design Data,Started,66.7
8,How to Design Functions,Started,28.6
9,Intro to Object Orientation,Started,14.3


In [22]:
melted_df["Module"].unique().tolist()

['Course Introduction',
 'How to Design Data',
 'How to Design Functions',
 'Intro to Object Orientation',
 'Data Hierarchies',
 'Designing Software']

In [23]:
# Create a horizontal bar chart using Plotly
fig = px.bar(melted_df, y='Module', x='Percentage Completion', color='Status', orientation='h',
             labels={'Percentage Completion': 'Percentage Completion (%)'},
             title='Percentage Completion by Students for Each Module',
            category_orders={'Module': sorted(melted_df['Module'].unique())})

fig.show()

In [24]:
data.head()

,completed_at,course_id,module_id,items_count,module_name,module_position,state,unlock_at,student_id,student_name,items_id,items_title,items_position,items_indent,items_type,items_module_id,item_cp_req_type,item_cp_req_completed,course_name
0,2019-06-25 22:29:00,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85224,Introduction Text,1,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
1,2019-06-25 22:29:00,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85225,Important Info Moving Foreward,2,0,Page,8135,must_mark_done,True,Canvas-Sandbox-Course
2,2019-06-25 22:29:00,2591,8135,3,Module 1: Course Introduction,1,completed,NaN,1001,student01,85262,Quiz 1 - Past Experience,3,1,Quiz,8135,must_submit,True,Canvas-Sandbox-Course
3,NaT,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85263,Designing Data,1,0,Page,8136,must_view,False,Canvas-Sandbox-Course
4,NaT,2591,8136,4,Module 2: How to Design Data,2,started,NaN,1001,student01,85228,Talk About It.,2,0,Discussion,8136,must_contribute,True,Canvas-Sandbox-Course


In [25]:
start_date="2019-06-15"
end_date="2019-07-15"

In [26]:
result_time = pd.DataFrame(columns=["Date", "Module", "Percentage Completion"])

In [27]:
type(start_date)

str

In [28]:
type(datetime.datetime.strptime(start_date, "%Y-%m-%d").date())

datetime.date

In [29]:
start_date=min( pd.to_datetime(data["completed_at"])).date()

In [30]:
type(start_date)

datetime.date

In [31]:
for module in module_dict.keys():
    timestamps = data[data.module_id.astype(str) == module][
        "completed_at"
    ].dt.date.unique()
    timestamps = [
        x for x in timestamps if type(x) != pd._libs.tslibs.nattype.NaTType
    ]

    # Filtering the timestamps based on the selected date range
    # start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    # end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()

In [32]:
timestamps = data[data.module_id.astype(str) == '8135'][
        "completed_at"
    ].dt.date.unique()
timestamps = [
        x for x in timestamps if type(x) != pd._libs.tslibs.nattype.NaTType
    ]

In [33]:
timestamps

[datetime.date(2019, 6, 25),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 6, 27)]

### Tab - Module Completion Over Time

In [34]:
#data[(data.state == 'completed') & (data.module_name == 'Module 6: Designing Software')].size

In [35]:
def get_completed_percentage_date(df, module, date):
    '''
    returns specifically the completed percentage of module in df till a specified date
    
    Inputs
    ------
    df: dataframe
    module: str, id of the module
    date: datetime.date, date till which the completion percentage of each module is desired
    
    Returns
    -------
    percentage, float, percentage value
    
    '''
    
    # Convert the date to datetime with time component set to midnight
    datetime_date = datetime.datetime.combine(date, datetime.datetime.min.time())
    
    df_module = df[df.module_id.astype(str) == module]
    total_module_students = df_module.student_id.unique().size
    
    # if there is not a sigle row with completion date then we get an datetime error since blanks are not compared to date 
    # to get around this edge case, return 0 
    if  df_module[df_module.state == 'completed'].size == 0:
        return 0.0
    
    percentage = df_module[(df_module.state=='completed') & (df_module['completed_at'].dt.date <= date)].student_id.unique().size / total_module_students 
    
    return percentage

In [36]:
# for every timestamp by date, for every module we need to compute the percentage completion for that module
# melted dataframe will have the columns: timestamp (date), module, status (optional), percentage completion

In [37]:
# fix how to remove NaT
timestamps = data['completed_at'].dt.date.unique()
timestamps = [x for x in timestamps if type(x) != pd._libs.tslibs.nattype.NaTType ]

In [38]:
timestamps

[datetime.date(2019, 6, 25),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 7, 9),
 datetime.date(2019, 6, 27)]

In [39]:
start_range = datetime.datetime.strptime('2019-07-01', '%Y-%m-%d').date()
end_range = datetime.datetime.strptime('2019-07-31', '%Y-%m-%d').date()

In [40]:
filtered_timestamps = [
    timestamp for timestamp in timestamps
    if start_range <= timestamp <= end_range
]

In [41]:
filtered_timestamps

[datetime.date(2019, 7, 9)]

In [42]:
## To build a dashboard lineplot with time on the x axis and the percentage completion on y axis
# for each module

result_time = pd.DataFrame(columns = ['Date', 'Module', 'Percentage Completion'])
for module in list(module_dict.keys()):
    timestamps = data[data.module_id.astype(str)==module]['completed_at'].dt.date.unique()
    timestamps = [x for x in timestamps if type(x) != pd._libs.tslibs.nattype.NaTType ]
    for date in timestamps:
        #print(module)
        value = round(get_completed_percentage_date(data, module, date)*100, 1)
        
        new_df = pd.DataFrame([[date, module_dict.get(module), value]], columns = ['Date', 'Module', 'Percentage Completion'])
        
        result_time = pd.concat([result_time, new_df], ignore_index = True)

In [43]:
result_time

,Date,Module,Percentage Completion
0,2019-06-25,Course Introduction,19.0
1,2019-06-26,Course Introduction,57.1
2,2019-06-27,Course Introduction,85.7
3,2019-07-09,How to Design Data,19.0
4,2019-06-26,How to Design Functions,14.3
5,2019-06-27,How to Design Functions,23.8
6,2019-06-26,Intro to Object Orientation,4.8
7,2019-06-27,Intro to Object Orientation,14.3
8,2019-06-26,Data Hierarchies,4.8
9,2019-06-27,Data Hierarchies,14.3


In [44]:
# Plotting the lineplot
fig2 = go.Figure()
for module, group in result_time.groupby('Module'):
    sorted_group = group.sort_values('Date')
    
    if len(sorted_group) == 1:
        fig2.add_trace(go.Scatter(x=sorted_group['Date'], y=sorted_group['Percentage Completion'], mode='markers', name=module))
        
    else:
        fig2.add_trace(go.Scatter(x=sorted_group['Date'], y=sorted_group['Percentage Completion'], mode='lines', name=module))

In [45]:
fig2.update_layout(
    title='Percentage Completion by Module',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Percentage')
)

# Set custom start and end dates for the x-axis
start_date = '2019-06-20'
end_date = '2019-07-20'
fig2.update_xaxes(range=[start_date, end_date])

# Specify custom spacing between dates on the x-axis
date_spacing = 'D7'  # Weekly spacing, adjust as per your requirement
fig2.update_xaxes(dtick=date_spacing)

fig2.show()

## Student Completion Per Item

In [46]:
module_dict.items()

dict_items([('8135', 'Course Introduction'), ('8136', 'How to Design Data'), ('8137', 'How to Design Functions'), ('8169', 'Intro to Object Orientation'), ('8170', 'Data Hierarchies'), ('8171', 'Designing Software')])

In [47]:
# Creating a dictionary of items per module
items_in_module = defaultdict(str)

for module in module_dict.keys():
    items_in_module[str(module)] = list(data[data.module_id.astype(str) == module].items_title.unique())

In [48]:
# Checking the items in a dictionary
items_in_module.get('8135')

['Introduction Text',
 'Important Info Moving Foreward',
 'Quiz 1 - Past Experience']

In [49]:
item_dict.items()

dict_items([('8135', 'Quiz 1 - Past Experience'), ('8136', 'Generic Quiz'), ('8137', 'Another Quiz'), ('8169', 'a2 - Object Orientation'), ('8170', 'Hardest Quiz Ever -- AVERAGE SCORE KEPT'), ('8171', 'Nobody Has Done This Item')])

In [50]:
item_dict.keys()

dict_keys(['8135', '8136', '8137', '8169', '8170', '8171'])

In [51]:
items_in_module.get('8135')

['Introduction Text',
 'Important Info Moving Foreward',
 'Quiz 1 - Past Experience']

In [52]:
data_module.items_title.str.strip() == 'Introduction Text'

16     False
17     False
18     False
19     False
36     False
       ...  
399    False
416    False
417    False
418    False
419    False
Name: items_title, Length: 84, dtype: bool

In [53]:
data_module.items_title.str.strip() == 'Another Quiz'

16     False
17     False
18     False
19     False
36     False
       ...  
399    False
416    False
417    False
418    False
419    False
Name: items_title, Length: 84, dtype: bool

In [54]:
# total student is give by 'total_students'

# result dataframe
student_completion_per_item = pd.DataFrame(columns = ['Module', 'Item', 'Item Percentage Completion', 'Item Position'])




# Computing the percentage completion in each item of a module
modules = list(data.module_id.unique().astype(str))

for module in modules:
    # number of student id related to Module
    data_module = data[(data.module_id.astype(str) == module)]
    

    for i, item in enumerate(items_in_module.get(module)):
        data_module_item = data_module[data_module.items_title == item]
        
        #print(data_module_item)
        
        item_percent_completion = round((data_module_item[data_module_item['item_cp_req_completed'] == True]['student_id'].unique().size)*100/total_students, 0)
        new_df = pd.DataFrame([[module_dict.get(module), item, item_percent_completion, i+1]], columns = ['Module', 'Item', 'Item Percentage Completion', 'Item Position'])
        
        student_completion_per_item = pd.concat([student_completion_per_item, new_df], ignore_index = True)

In [55]:
student_completion_per_item.head()

,Module,Item,Item Percentage Completion,Item Position
0,Course Introduction,Introduction Text,100.0,1
1,Course Introduction,Important Info Moving Foreward,100.0,2
2,Course Introduction,Quiz 1 - Past Experience,86.0,3
3,How to Design Data,Designing Data,62.0,1
4,How to Design Data,Talk About It.,48.0,2


In [56]:
# Plotting

# Group the DataFrame by 'module'
grouped_df = student_completion_per_item.groupby('Module')


# Calculate the number of rows and columns for the subplots
num_modules = len(grouped_df)
num_cols = 2
num_rows = (num_modules + num_cols - 1) // num_cols

# Create subplots with multiple columns
fig3 = make_subplots(rows=num_rows, cols=num_cols, shared_xaxes=True, vertical_spacing=0.01)


# Define custom colors for the bars
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# Iterate over each module group
for i, (module, group) in enumerate(grouped_df):
    # Calculate the subplot row and column based on the index
    row = i // num_cols + 1
    col = i % num_cols + 1
    
    # Create a horizontal bar chart for the module
    fig3.add_trace(go.Bar(y=group['Item'][::-1],
                          x=group['Item Percentage Completion'][::-1],
                          orientation='h',
                          name=module,
                          marker=dict(opacity=0.8),
                          text=[],
                          hovertemplate="Item Position: %{text}<br>Item Title: %{y}<br>Completion:%{x}%<extra></extra>"),
                   row=row,
                   col=col)

# Update the layout of the figure
fig3.update_layout(height=200*len(grouped_df), title='Percentage Completion by Item for Each Module',
                  xaxis_title='Percentage Completion', yaxis_title='Item')


# Show the figure
fig3.show()

In [57]:
student_completion_per_item

,Module,Item,Item Percentage Completion,Item Position
0,Course Introduction,Introduction Text,100.0,1
1,Course Introduction,Important Info Moving Foreward,100.0,2
2,Course Introduction,Quiz 1 - Past Experience,86.0,3
3,How to Design Data,Designing Data,62.0,1
4,How to Design Data,Talk About It.,48.0,2
5,How to Design Data,a1 - Data Assignment,67.0,3
6,How to Design Data,Generic Quiz,52.0,4
7,How to Design Functions,Designing Functions,33.0,1
8,How to Design Functions,This is a pdf.pdf,43.0,2
9,How to Design Functions,Another Quiz,24.0,3


In [58]:
items_in_module.get('8138')

## Alternative approach with data augmentation and new visualizations

In [163]:
from dash import dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import dash
from dash import dash_table
from dash.dependencies import Input, Output, State
import re

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from collections import defaultdict

from datetime import *
import datetime

pio.renderers.default = "iframe"

In [164]:
## helper functions

def remove_special_characters(string):
    # Define the pattern for special characters
    pattern = r'[^a-zA-Z0-9]'

    # Use regex to remove special characters
    cleaned_string = re.sub(pattern, '', string)

    return cleaned_string

In [165]:
# reading the data
data = pd.read_csv("../data/module_data_augmented.csv")

In [166]:
data.head(2)

,completed_at,course_id,module_id,items_count,module_name,module_position,state,unlock_at,student_id,student_name,items_id,items_title,items_position,items_indent,items_type,items_module_id,item_cp_req_type,item_cp_req_completed,course_name
0,10-05-2023 21:23,121654,787819,1,Get Started with the Canvas API,2,completed,NaN,93997,LtsStudentFive,5619042,CAPICO-API-usage-guidelines-v1.1.pdf,2,0,File,787819,NaN,NaN,🤖 Learning Services Student API Sandbox
1,10-05-2023 21:23,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618861,Procedures for Running Scripts,1,1,Page,787806,NaN,NaN,🤖 Learning Services Student API Sandbox


In [167]:
# dtype conversion
categorical_cols = [
    "course_id",
    "module_id",
    "module_name",
    "state",
    "student_id",
    "student_name",
    "items_id",
    "items_title",
    "items_type",
    "items_module_id",
    "item_cp_req_type",
    "item_cp_req_completed",
    "course_name",
]

# convert the timestamp to datetime format
# fix the column data types
data["completed_at"] = pd.to_datetime(data["completed_at"],format = "%d-%m-%Y %H:%M")

for col in categorical_cols:
    data[col] = data[col].astype("category")
    
# remove special characters from course_name
data["course_name"] = data["course_name"].apply(remove_special_characters)

In [168]:
# Make the mapping of any id to the corresponding names
# Make a dictionary of ids and their names
module_num, module_dict, item_dict, course_dict, student_dict = (defaultdict(str) for _ in range(5))

for _, row in data.iterrows():
    module_dict[str(row["module_id"])] = re.sub(
        r"^Module\s+\d+:\s+", "", row["module_name"]
    )
    item_dict[str(row["items_module_id"])] = row["items_title"]
    course_dict[str(row["course_id"])] = row["course_name"]
    student_dict[str(row["student_id"])] = row["student_name"]

## Rememeber use callbacks to filter the original dataframe to the subset dataframe you want to make computations, assume each helper function gets the requisite subset dataframe as  input

In [169]:
# get the unique course list
def get_course_list(df):
    '''
    returns the course list
    '''
    course_list = list(course_dict[str(x)] for x in df.course_id.unique())
    return course_list

In [170]:
# get the unique module list under a specific course - Check if this 
def get_module_list(df):
    '''
    returns the module list for the df. df is a subset dataframe that is already filtered to a single course
    '''
    module_list = list(module_dict[str(x)] for x in df.module_id.unique())
    return module_list

In [171]:
def get_item_list(df):
    '''
    returns the item list for a given module in the course. df is a subset dataframe that is already filtered to a single course
    '''
    item_list = list(item_dict[str(x)] for x in df.items_module_id.unique())
    return item_list

In [172]:
def get_student_list(df):
    '''
    returns the student list for a given module in the course. df is a subset dataframe that is already filtered to a single course
    '''
    student_list = list(student_dict[str(x)] for x in df.student_id.unique())
    return student_list

In [173]:
temp_data = data[data.course_name.astype(str)==get_course_list(data)[0]]
get_student_list(temp_data)

[' LtsStudentFive',
 ' LtsStudentFour',
 ' LtsStudentThree',
 'LTS StudentOne',
 'LTS StudentTwo']

# Manual Fitlering the selections

In [174]:
selected_course = get_course_list(data)[0]
selected_course

'LearningServicesStudentAPISandbox'

In [175]:
selected_modules = get_module_list(data[data["course_name"]==selected_course])
selected_modules

['Get Started with the Canvas API',
 'Scripting (current script instructions for LS Ops - to be improved!) ',
 'Data Science Introduction',
 'Data Science Disciplines',
 'Probability and Statistics']

### Computing the duration (days) for completion of modules within a course by a student


In [176]:
# data filered by selected course
data_filteredby_course = data[data.course_name.astype(str)==get_course_list(data)[0]]

In [177]:
# filter by module state as completed
data_filteredby_state = data_filteredby_course[data_filteredby_course.state == 'completed']
data_filteredby_state.head()

,completed_at,course_id,module_id,items_count,module_name,module_position,state,unlock_at,student_id,student_name,items_id,items_title,items_position,items_indent,items_type,items_module_id,item_cp_req_type,item_cp_req_completed,course_name
0,2023-05-10 21:23:00,121654,787819,1,Get Started with the Canvas API,2,completed,NaN,93997,LtsStudentFive,5619042,CAPICO-API-usage-guidelines-v1.1.pdf,2,0,File,787819,NaN,NaN,LearningServicesStudentAPISandbox
1,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618861,Procedures for Running Scripts,1,1,Page,787806,NaN,NaN,LearningServicesStudentAPISandbox
2,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618862,Canvas Blueprint Script,2,1,Page,787806,NaN,NaN,LearningServicesStudentAPISandbox
3,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618863,How to Launch Jupyter Notebooks,3,1,File,787806,NaN,NaN,LearningServicesStudentAPISandbox
4,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618864,GitHub,4,1,Page,787806,NaN,NaN,LearningServicesStudentAPISandbox


In [178]:
# Create column that computes the number of days from the set date say 1-Jan-2023

In [179]:
data_filteredby_state['completed_at'].head()

0   2023-05-10 21:23:00
1   2023-05-10 21:23:00
2   2023-05-10 21:23:00
3   2023-05-10 21:23:00
4   2023-05-10 21:23:00
Name: completed_at, dtype: datetime64[ns]

In [180]:
course_start_date = datetime.datetime(2023, 1, 1)
course_start_time = datetime.time(1, 30)
course_combined_datetime = course_start_date.combine(course_start_date, course_start_time)
print(course_combined_datetime)

2023-01-01 01:30:00


In [181]:
# difference in days
(combined_datetime - data_filteredby_state['completed_at'])/np.timedelta64(1, 'D')

0      82.171528
1      82.171528
2      82.171528
3      82.171528
4      82.171528
23     68.198611
24     68.198611
25     68.198611
26     68.198611
27     68.198611
46     75.325694
47     75.325694
48     75.325694
49     75.325694
50     75.325694
51     68.197222
52     68.197222
53     68.197222
54     68.197222
55     68.197222
56     68.197222
57     68.197222
69     75.325694
70     75.325694
71     75.325694
72     75.325694
73     75.325694
92     75.325694
93     75.325694
94     75.325694
95     75.325694
96     75.325694
97     69.272917
98     69.272917
99     69.272917
100    69.272917
101    69.272917
102    69.272917
103    69.272917
104    69.272222
105    69.272222
106    69.272222
107    69.272222
108    69.272222
109    69.272222
110    68.198611
111    68.198611
112    68.198611
113    68.198611
114    68.198611
Name: completed_at, dtype: float64

In [182]:
data_filteredby_state['duration'] = round((combined_datetime - data_filteredby_state['completed_at'])/np.timedelta64(1, 'D'), 0)

C:\Users\ranji\AppData\Local\Temp\ipykernel_26508\3389343505.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [183]:
data_filteredby_state['duration']

0      82.0
1      82.0
2      82.0
3      82.0
4      82.0
23     68.0
24     68.0
25     68.0
26     68.0
27     68.0
46     75.0
47     75.0
48     75.0
49     75.0
50     75.0
51     68.0
52     68.0
53     68.0
54     68.0
55     68.0
56     68.0
57     68.0
69     75.0
70     75.0
71     75.0
72     75.0
73     75.0
92     75.0
93     75.0
94     75.0
95     75.0
96     75.0
97     69.0
98     69.0
99     69.0
100    69.0
101    69.0
102    69.0
103    69.0
104    69.0
105    69.0
106    69.0
107    69.0
108    69.0
109    69.0
110    68.0
111    68.0
112    68.0
113    68.0
114    68.0
Name: duration, dtype: float64

In [184]:
data_filteredby_state.head()

,completed_at,course_id,module_id,items_count,module_name,module_position,state,unlock_at,student_id,student_name,items_id,items_title,items_position,items_indent,items_type,items_module_id,item_cp_req_type,item_cp_req_completed,course_name,duration
0,2023-05-10 21:23:00,121654,787819,1,Get Started with the Canvas API,2,completed,NaN,93997,LtsStudentFive,5619042,CAPICO-API-usage-guidelines-v1.1.pdf,2,0,File,787819,NaN,NaN,LearningServicesStudentAPISandbox,82.0
1,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618861,Procedures for Running Scripts,1,1,Page,787806,NaN,NaN,LearningServicesStudentAPISandbox,82.0
2,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618862,Canvas Blueprint Script,2,1,Page,787806,NaN,NaN,LearningServicesStudentAPISandbox,82.0
3,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618863,How to Launch Jupyter Notebooks,3,1,File,787806,NaN,NaN,LearningServicesStudentAPISandbox,82.0
4,2023-05-10 21:23:00,121654,787806,4,Scripting (current script instructions for LS ...,3,completed,NaN,93997,LtsStudentFive,5618864,GitHub,4,1,Page,787806,NaN,NaN,LearningServicesStudentAPISandbox,82.0


In [185]:
# Next we want to keep only one unique row per student, thereby there are no repetition for the same student. 
# This is important because repetitiion will influence the percentile computation
# Remember for the same moduel there are multiple rows equivalent to the number of items within the module.

data_filteredby_state = data_filteredby_state[['module_id', 'module_name', 'state', 'duration', 'student_id']].drop_duplicates()

In [186]:
# Ensure that module id is considered as string and not a number
data_filteredby_state['module_id'] = data_filteredby_state['module_id'].astype(str)

In [188]:
# Make the boxp lot

# Create the box plot using Plotly Express
fig = px.box(data_filteredby_state, x='module_id', y='duration', points='all', title='Module Completion Duration',
             hover_data=['duration'])

# Update the box plot to show mean and percentiles
fig.update_traces(boxpoints='outliers', boxmean=True) #, meanline_visible=True)

# Show the plot
fig.show()


In [200]:
data_filteredby_state.sort_values('student_id')

,module_id,module_name,state,duration,student_id
69,787819,Get Started with the Canvas API,completed,75.0,93991
70,787806,Scripting (current script instructions for LS ...,completed,75.0,93991
92,787819,Get Started with the Canvas API,completed,75.0,93992
93,787806,Scripting (current script instructions for LS ...,completed,75.0,93992
97,790496,Data Science Introduction,completed,69.0,93992
104,790743,Data Science Disciplines,completed,69.0,93992
110,790744,Probability and Statistics,completed,68.0,93992
46,787819,Get Started with the Canvas API,completed,75.0,93994
47,787806,Scripting (current script instructions for LS ...,completed,75.0,93994
51,790496,Data Science Introduction,completed,68.0,93994


In [198]:
print(data_filteredby_state.groupby(['student_id', 'module_id'])['duration'])